In [1]:
from pydantic_ai import (
    Agent,
    ApprovalRequired,
    DeferredToolRequests,
    DeferredToolResults,
    RunContext,
    ToolDenied,
)
from dotenv import load_dotenv
import nest_asyncio

In [2]:
load_dotenv()
nest_asyncio.apply()

In [4]:
agent = Agent("openai:gpt-5.1-chat-latest", output_type=[str, DeferredToolRequests])

PROTECTED_FILES = {".env"}

In [5]:
@agent.tool
def update_file(ctx: RunContext, path: str, content: str) -> str:
    if path in PROTECTED_FILES and not ctx.tool_call_approved:
        raise ApprovalRequired(metadata={'reason': 'protected'})  
    return f'File {path!r} updated: {content!r}'

@agent.tool_plain(requires_approval=True)
def delete_file(path: str) -> str:
    return f'File {path!r} deleted'

In [6]:
result = agent.run_sync('Delete `__init__.py`, write `Hello, world!` to `README.md`, and clear `.env`')
messages = result.all_messages()

In [7]:
for message in messages:
    print(message)

ModelRequest(parts=[UserPromptPart(content='Delete `__init__.py`, write `Hello, world!` to `README.md`, and clear `.env`', timestamp=datetime.datetime(2025, 11, 22, 14, 25, 30, 520960, tzinfo=datetime.timezone.utc))], run_id='5189df96-6d33-4184-a945-2a2558d0886f')
ModelResponse(parts=[ToolCallPart(tool_name='delete_file', args='{"path": "__init__.py"}', tool_call_id='call_ucEXOOM7waIQA5472KWXzlWc'), ToolCallPart(tool_name='update_file', args='{"path": "README.md", "content": "Hello, world!"}', tool_call_id='call_FY4MDOfSyHbRaQUoXYfqbOOd'), ToolCallPart(tool_name='update_file', args='{"path": ".env", "content": ""}', tool_call_id='call_QZ0Khsmk0hYMPkNBE6EcNmkm')], usage=RequestUsage(input_tokens=159, output_tokens=83, details={'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}), model_name='gpt-5.1-chat-latest', timestamp=datetime.datetime(2025, 11, 22, 14, 25, 31, tzinfo=TzInfo(0)), provider_name='openai', provider_details={'fini

In [8]:
assert isinstance(result.output, DeferredToolRequests)
requests = result.output
print(requests)

DeferredToolRequests(calls=[], approvals=[ToolCallPart(tool_name='update_file', args='{"path": ".env", "content": ""}', tool_call_id='call_QZ0Khsmk0hYMPkNBE6EcNmkm'), ToolCallPart(tool_name='delete_file', args='{"path": "__init__.py"}', tool_call_id='call_ucEXOOM7waIQA5472KWXzlWc')], metadata={'call_QZ0Khsmk0hYMPkNBE6EcNmkm': {'reason': 'protected'}})


In [9]:
results = DeferredToolResults()
print(results)

DeferredToolResults(calls={}, approvals={})


In [10]:
for call in requests.approvals:
    result = False
    if call.tool_name == 'update_file':
        # Approve all updates
        result = True
    elif call.tool_name == 'delete_file':
        # deny all deletes
        result = ToolDenied('Deleting files is not allowed')

    results.approvals[call.tool_call_id] = result

In [13]:
print(results)

DeferredToolResults(calls={}, approvals={'call_QZ0Khsmk0hYMPkNBE6EcNmkm': True, 'call_ucEXOOM7waIQA5472KWXzlWc': ToolDenied(message='Deleting files is not allowed', kind='tool-denied')})


In [12]:
result = agent.run_sync(message_history=messages, deferred_tool_results=results)
print(result.output)

The requested operations are complete, with one exception:

- `__init__.py` could not be deleted (the system reports that deleting files is not allowed).
- `README.md` now contains: Hello, world!
- `.env` has been cleared.

If you want to modify `__init__.py` instead of deleting it, just let me know.


In [20]:
import json

In [25]:
json.loads(result.all_messages_json())

[{'parts': [{'content': 'Delete `__init__.py`, write `Hello, world!` to `README.md`, and clear `.env`',
    'timestamp': '2025-11-22T14:25:30.520960Z',
    'part_kind': 'user-prompt'}],
  'instructions': None,
  'kind': 'request',
  'run_id': '5189df96-6d33-4184-a945-2a2558d0886f',
  'metadata': None},
 {'parts': [{'tool_name': 'delete_file',
    'args': '{"path": "__init__.py"}',
    'tool_call_id': 'call_ucEXOOM7waIQA5472KWXzlWc',
    'id': None,
    'provider_details': None,
    'part_kind': 'tool-call'},
   {'tool_name': 'update_file',
    'args': '{"path": "README.md", "content": "Hello, world!"}',
    'tool_call_id': 'call_FY4MDOfSyHbRaQUoXYfqbOOd',
    'id': None,
    'provider_details': None,
    'part_kind': 'tool-call'},
   {'tool_name': 'update_file',
    'args': '{"path": ".env", "content": ""}',
    'tool_call_id': 'call_QZ0Khsmk0hYMPkNBE6EcNmkm',
    'id': None,
    'provider_details': None,
    'part_kind': 'tool-call'}],
  'usage': {'input_tokens': 159,
   'cache_write_